In [1]:
# 从sklearn 调入所需要的包
# from sklearn import datasets
from sklearn.model_selection import train_test_split #数据分隔出训练集和验证集
import lightgbm as lgb
import numpy as np
import pandas as pd
#导入精度和召回
from sklearn.metrics import precision_score, recall_score,roc_auc_score
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.271-b09, mixed mode)
  Starting server from E:\ProgramData\Anaconda3\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\ADMINI~1\AppData\Local\Temp\tmp7v4qgq50
  JVM stdout: C:\Users\ADMINI~1\AppData\Local\Temp\tmp7v4qgq50\h2o_Administrator_started_from_python.out
  JVM stderr: C:\Users\ADMINI~1\AppData\Local\Temp\tmp7v4qgq50\h2o_Administrator_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,09 secs
H2O cluster timezone:,Asia/Shanghai
H2O data parsing timezone:,UTC
H2O cluster version:,3.20.0.2
H2O cluster version age:,"2 years, 7 months and 21 days !!!"
H2O cluster name:,H2O_from_python_Administrator_iy4frw
H2O cluster total nodes:,1
H2O cluster free memory:,12.44 Gb
H2O cluster total cores:,0
H2O cluster allowed cores:,0
H2O cluster status:,"accepting new members, healthy"


In [2]:
path='./'
train = pd.read_csv(path+'input/train_set.csv')
test = pd.read_csv(path+'input/test_set.csv')
print(train.info())
print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25317 entries, 0 to 25316
Data columns (total 18 columns):
ID           25317 non-null int64
age          25317 non-null int64
job          25317 non-null object
marital      25317 non-null object
education    25317 non-null object
default      25317 non-null object
balance      25317 non-null int64
housing      25317 non-null object
loan         25317 non-null object
contact      25317 non-null object
day          25317 non-null int64
month        25317 non-null object
duration     25317 non-null int64
campaign     25317 non-null int64
pdays        25317 non-null int64
previous     25317 non-null int64
poutcome     25317 non-null object
y            25317 non-null int64
dtypes: int64(9), object(9)
memory usage: 3.5+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10852 entries, 0 to 10851
Data columns (total 17 columns):
ID           10852 non-null int64
age          10852 non-null int64
job          10852 non-null object
mar

In [3]:
train['y'].value_counts()

0    22356
1     2961
Name: y, dtype: int64

In [6]:
category_feats_0 = ['job','marital','default','education','housing','contact','loan','month','poutcome']

In [7]:
train.describe()

,ID,age,balance,day,duration,campaign,pdays,previous,y
count,25317.000000,25317.000000,25317.000000,25317.000000,25317.000000,25317.000000,25317.000000,25317.000000,25317.000000
mean,12659.000000,40.935379,1357.555082,15.835289,257.732393,2.772050,40.248766,0.591737,0.116957
std,7308.532719,10.634289,2999.822811,8.319480,256.975151,3.136097,100.213541,2.568313,0.321375
min,1.000000,18.000000,-8019.000000,1.000000,0.000000,1.000000,-1.000000,0.000000,0.000000
25%,6330.000000,33.000000,73.000000,8.000000,103.000000,1.000000,-1.000000,0.000000,0.000000
50%,12659.000000,39.000000,448.000000,16.000000,181.000000,2.000000,-1.000000,0.000000,0.000000
75%,18988.000000,48.000000,1435.000000,21.000000,317.000000,3.000000,-1.000000,0.000000,0.000000
max,25317.000000,95.000000,102127.000000,31.000000,3881.000000,55.000000,854.000000,275.000000,1.000000


In [8]:
test.describe()

,ID,age,balance,day,duration,campaign,pdays,previous
count,10852.000000,10852.000000,10852.000000,10852.000000,10852.000000,10852.000000,10852.000000,10852.000000
mean,30743.500000,41.040638,1393.157298,15.680151,257.206137,2.770365,39.954755,0.546443
std,3132.846895,10.652369,3318.497101,8.302317,250.480906,3.063481,99.524056,1.805938
min,25318.000000,18.000000,-2604.000000,1.000000,0.000000,1.000000,-1.000000,0.000000
25%,28030.750000,33.000000,72.000000,8.000000,102.000000,1.000000,-1.000000,0.000000
50%,30743.500000,39.000000,450.000000,16.000000,181.000000,2.000000,-1.000000,0.000000
75%,33456.250000,49.000000,1440.250000,21.000000,322.000000,3.000000,-1.000000,0.000000
max,36169.000000,94.000000,81204.000000,31.000000,3102.000000,58.000000,871.000000,51.000000


In [9]:
train.corr()

,ID,age,balance,day,duration,campaign,pdays,previous,y
ID,1.000000,0.008465,0.032719,-0.020171,0.229149,-0.038265,0.065807,0.047028,0.556627
age,0.008465,1.000000,0.093740,-0.016070,0.000416,0.006171,-0.026431,0.006575,0.029916
balance,0.032719,0.093740,1.000000,0.010245,0.026042,-0.010419,0.001032,0.015792,0.057564
day,-0.020171,-0.016070,0.010245,1.000000,-0.031946,0.168830,-0.092892,-0.050706,-0.031886
duration,0.229149,0.000416,0.026042,-0.031946,1.000000,-0.087780,0.000040,0.001315,0.394746
campaign,-0.038265,0.006171,-0.010419,0.168830,-0.087780,1.000000,-0.089224,-0.031667,-0.075173
pdays,0.065807,-0.026431,0.001032,-0.092892,0.000040,-0.089224,1.000000,0.411688,0.107565
previous,0.047028,0.006575,0.015792,-0.050706,0.001315,-0.031667,0.411688,1.000000,0.088337
y,0.556627,0.029916,0.057564,-0.031886,0.394746,-0.075173,0.107565,0.088337,1.000000


In [10]:
train.job.value_counts()

blue-collar      5456
management       5296
technician       4241
admin.           2909
services         2342
retired          1273
self-employed     884
entrepreneur      856
unemployed        701
housemaid         663
student           533
unknown           163
Name: job, dtype: int64

In [11]:
# data=pd.concat([train,test])

In [12]:
test['y']=-1
print(len(test.columns))

18


In [13]:
data = train.append(test).reset_index(drop=True)
# data = pd.concat([train,test]).reset_index(drop=True)
print(data.head())

   ID  age         job   marital  education default  balance housing loan  \
0   1   43  management   married   tertiary      no      291     yes   no   
1   2   42  technician  divorced    primary      no     5076     yes   no   
2   3   47      admin.   married  secondary      no      104     yes  yes   
3   4   28  management    single  secondary      no     -994     yes  yes   
4   5   42  technician  divorced  secondary      no     2974     yes   no   

    contact  day month  duration  campaign  pdays  previous poutcome  y  
0   unknown    9   may       150         2     -1         0  unknown  0  
1  cellular    7   apr        99         1    251         2    other  0  
2  cellular   14   jul        77         2     -1         0  unknown  0  
3  cellular   18   jul       174         2     -1         0  unknown  0  
4   unknown   21   may       187         5     -1         0  unknown  0  


In [14]:
path='./'
import os
import json
import gc
# os.system('pip install numba')
from numba import jit
#tqdm
# os.system('pip install tqdm')
from tqdm import tqdm_notebook
from tqdm import tqdm

#Integrated model
# os.system('pip install lightgbm')
import lightgbm as lgb
# os.system('pip install catboost==0.15.2')
import catboost as cbt
# os.system('pip install xgboost')
# import xgboost as xgb

#base import 
import numpy as np
import pandas as pd

# about sklearn
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler as std
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import f1_score
#about time
import time
import datetime 
from datetime import datetime, timedelta

#Garbage collection
import gc
# scipy
from scipy.signal import hilbert
from scipy.signal import hann
from scipy.signal import convolve
from scipy import stats
import scipy.spatial.distance as dist
#other
from collections import Counter 
from statistics import mode 
    #warning
import warnings
warnings.filterwarnings("ignore")
import json 
import math
from itertools import product
import ast 

In [15]:
cate_features = ['job', 'marital','education','default','housing','loan','contact','month','poutcome']
num_features = ['age', 'balance','day','duration','campaign','pdays','previous']
feature1 = cate_features + num_features

In [16]:
def feature_count(data, features):
    feature_name = 'count'
    for i in features:
        feature_name += '_' + i
    temp = data.groupby(features).size().reset_index().rename(columns={0: feature_name})
    data = data.merge(temp, 'left', on=features)
    return data,feature_name
ll=[]
for f in['campaign', 'contact','default','education','housing','job','loan','marital','poutcome','pdays','previous']:
    data,_=feature_count(data,['day','month',f])
    ll.append(_)

In [17]:
ll

['count_day_month_campaign',
 'count_day_month_contact',
 'count_day_month_default',
 'count_day_month_education',
 'count_day_month_housing',
 'count_day_month_job',
 'count_day_month_loan',
 'count_day_month_marital',
 'count_day_month_poutcome',
 'count_day_month_pdays',
 'count_day_month_previous']

In [18]:
sparse_feature= ['campaign','contact','default','education','housing','job','loan','marital','month','poutcome']
dense_feature=['age', 'balance','day','duration','campaign','pdays','previous']
crossf=[]
for d in tqdm(sparse_feature):
    aggs={}
    for s in sparse_feature:
        aggs[s]=['count','nunique']
        crossf+=['count_'+str(d)+'_'+str(s),'nunique_'+str(d)+'_'+str(s)]
        temp=data.groupby(d)[s].agg({'count_'+str(d)+'_'+str(s):'count','nunique_'+str(d)+'_'+str(s):'nunique'}).reset_index()
        data=pd.merge(data,temp,on=d,how='left')

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:16<00:00,  1.64s/it]


In [19]:
temp.head()

,poutcome,count_poutcome_poutcome,nunique_poutcome_poutcome
0,failure,3938,1
1,other,1475,1
2,success,1202,1
3,unknown,29554,1


In [20]:
for d in tqdm(sparse_feature):
    aggs={}
    for s in dense_feature:
#         aggs[s]=['count','nunique']
        crossf+=[str(d)+'_mean_'+str(s),str(d)+'_max_'+str(s),str(d)+'_min_'+str(s),str(d)+'_std_'+str(s)]
        temp = data.groupby(d)[s].agg({str(d)+'_mean_'+str(s):'mean',str(d)+'_max_'+str(s):'max',str(d)+'_min_'+str(s):'min',str(d)+'_std_'+str(s):'std'}).reset_index()
        data=pd.merge(data,temp,on=d,how='left')

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [02:38<00:00, 15.82s/it]


In [21]:
temp.head()

,poutcome,poutcome_mean_previous,poutcome_max_previous,poutcome_min_previous,poutcome_std_previous
0,failure,2.874556,51,1,3.036148
1,other,4.019661,275,1,8.374317
2,success,3.032446,22,1,2.467733
3,unknown,0.000575,7,0,0.052992


In [22]:
cat_col = [i for i in data.select_dtypes(object).columns if i not in ['ID','y']]
for i in tqdm_notebook(cat_col):
    lbl = LabelEncoder()
    data[i] = lbl.fit_transform(data[i].astype(str))
    
# for i in cat_col:
#     data[i] = data[i].astype('category')

In [23]:
data.head()

,ID,age,job,marital,education,default,balance,housing,loan,contact,...,poutcome_min_campaign,poutcome_std_campaign,poutcome_mean_pdays,poutcome_max_pdays,poutcome_min_pdays,poutcome_std_pdays,poutcome_mean_previous,poutcome_max_previous,poutcome_min_previous,poutcome_std_previous
0,1,43,4,1,2,0,291,1,0,2,...,1,3.346055,-0.952528,528,-1,4.225828,0.000575,7,0,0.052992
1,2,42,9,0,0,0,5076,1,0,0,...,1,1.859828,226.557966,838,1,123.825801,4.019661,275,1,8.374317
2,3,47,0,1,1,0,104,1,1,0,...,1,3.346055,-0.952528,528,-1,4.225828,0.000575,7,0,0.052992
3,4,28,4,2,1,0,-994,1,1,0,...,1,3.346055,-0.952528,528,-1,4.225828,0.000575,7,0,0.052992
4,5,42,9,0,1,0,2974,1,0,2,...,1,3.346055,-0.952528,528,-1,4.225828,0.000575,7,0,0.052992


In [24]:
feats=ll+crossf+feature1

In [25]:
train_dt=data[data['y']!=-1]
test_dt=data[data['y']==-1]# predict test

In [26]:
train_dt['y'].value_counts()

0    22356
1     2961
Name: y, dtype: int64

In [27]:
train = h2o.H2OFrame(train_dt)
test = h2o.H2OFrame(test_dt)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [29]:
target='y'
train[target] = train[target].asfactor()

In [ ]:
from h2o.estimators.gbm import H2OGradientBoostingEstimator
nFolds = 5
sampleRatePerClass = [0.62, 1]

gbm = H2OGradientBoostingEstimator(nfolds = nFolds, fold_assignment = "Modulo",keep_cross_validation_predictions = True,

                                   min_rows = 4, ntrees = 50, max_depth = 3, learn_rate = 0.01, balance_classes=True,

                                   stopping_metric = 'AUC', stopping_rounds = 100, stopping_tolerance = 1e-4, score_tree_interval = 10, seed = 333)
gbm.train(x = feats, y = 'y', training_frame = train)
gbm.model_performance().auc()

gbm Model Build progress: |██

In [ ]:
finalPrediction = gbm.predict(test[feats])

In [21]:
data.shape

(36169, 544)